<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V9_pureData_SMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [17]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

外部資料讀入

In [18]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NFAhP8NUVsxzEq55siFA0yHvnXY5GWqiKGSOKC4y1Qg/edit?usp=sharing')
worksheet = gsheets.worksheet("external_validation_1014")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall,BW_first,BW_last,BW_diff_seq
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,74.0,74.1,0.1
1,1322,NaN,NaN,NaN,0,327,1,92,0,10,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,75.3,69.2,-6.1
2,1319,NaN,NaN,NaN,0,255,1,78,1,5,...,-1.0,NaN,0.0,0.0,0.0,0.0,0.0,46.0,35.7,-10.3
3,1333,NaN,NaN,NaN,0,293,1,82,1,10,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,58.7,53.4,-5.3
4,1452,NaN,NaN,NaN,0,341,0,80,1,10,...,-1.0,NaN,0.0,0.0,0.0,0.0,0.0,40.7,36.5,-4.2


In [19]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,6216.0,1391.613256,770.503700,4.0,1187.000,1267.000,1388.250000,20295.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,6216.0,0.286519,0.452171,0.0,0.000,0.000,1.000000,1.000000
觀察天數,6216.0,208.509492,110.248977,0.0,121.000,232.000,297.000000,365.000000
性別_is_male,6216.0,0.510457,0.499931,0.0,0.000,1.000,1.000000,1.000000
預估年齡,6216.0,78.604086,11.724613,1.0,72.000,81.000,87.000000,124.000000
DNR_flag,6216.0,0.455598,0.498065,0.0,0.000,0.000,1.000000,1.000000
ADL_總分_max,6216.0,28.906853,31.175410,0.0,0.000,20.000,50.000000,100.000000


In [20]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
DNR_flag,0,0.0000
ADL_總分_max,0,0.0000
ADL_明顯惡化,0,0.0000
六個月內住院次數,0,0.0000
ADL_last_CouldNot,0,0.0000


訓練資料讀入

In [21]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1qljyp9lq3QsZ7O2O7FQxm7taEWQi3F3bZgNMcQ7NJeE/edit?usp=sharing')
worksheet = gsheets.worksheet("training_data_1014")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall,BW_first,BW_last,BW_diff_seq
0,1325,NaN,NaN,NaN,0,739,1,66,1,0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,55.50,51.00,-4.50
1,1160,NaN,NaN,NaN,0,788,1,89,1,95,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,50.25,46.70,-3.55
2,1253,NaN,NaN,NaN,0,1292,0,89,0,60,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,63.10,64.05,0.95
3,1342,NaN,NaN,NaN,0,584,0,93,1,30,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,55.50,52.65,-2.85
4,1343,NaN,NaN,NaN,0,583,1,90,0,90,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,66.85,65.60,-1.25


In [22]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,23901.0,1457.124723,3207.707525,1.0,1174.0,1233.0,1327.000000,100463.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,23901.0,0.220577,0.414644,0.0,0.0,0.0,0.000000,1.000000
觀察天數,23901.0,584.830007,428.471637,0.0,211.0,509.0,957.000000,1460.000000
性別_is_male,23901.0,0.491444,0.499937,0.0,0.0,0.0,1.000000,1.000000
預估年齡,23901.0,79.317476,11.842274,0.0,72.0,82.0,88.000000,125.000000
DNR_flag,23901.0,0.409062,0.491671,0.0,0.0,0.0,1.000000,1.000000
ADL_總分_max,23901.0,31.028200,32.745984,0.0,0.0,20.0,55.000000,100.000000


In [23]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
DNR_flag,0,0.0000
ADL_總分_max,0,0.0000
ADL_明顯惡化,0,0.0000
六個月內住院次數,0,0.0000
ADL_last_CouldNot,0,0.0000


In [24]:
features = df_missing_info[df_missing_info['Missing Ratio']<0.3].index.tolist()

In [25]:
features

['H01_NUM',
 '性別_is_male',
 '觀察天數',
 '死亡標記',
 '預估年齡',
 'DNR_flag',
 'ADL_總分_max',
 'ADL_明顯惡化',
 '六個月內住院次數',
 'ADL_last_CouldNot',
 'ADL_first_CouldNot',
 'ADL_first_score',
 'ADL_Max',
 'ADL_Min',
 'last_has_denture',
 'diff_has_denture',
 'first_has_denture',
 'first_has_feeding_tube',
 'last_has_feeding_tube',
 'diff_has_feeding_tube',
 'last_ 意識總分',
 '意識總分Max',
 '意識總分_diff',
 'had_fall',
 '使用呼吸輔具',
 'first_ 意識總分',
 'BW_first',
 'BW_last',
 'BW_diff_seq',
 'ADL_std',
 'ADL_last_score',
 'ADL_diff_seq']

In [26]:
dfNew = df[features]

In [27]:
dfNew = dfNew.fillna(0)

In [28]:
import numpy as np
import pandas as pd

def compute_smd_by_death(df, group_col='死亡標記', group1=1, group0=0, drop_id_cols=None):
    if drop_id_cols is None:
        drop_id_cols = ['H01_NUM']  # 可以再加別的 ID 欄位進來

    # 只抓數值欄位
    num_cols = df.select_dtypes(include='number').columns.tolist()

    # 移除分組欄位與 ID 欄位
    for col in [group_col] + drop_id_cols:
        if col in num_cols:
            num_cols.remove(col)

    smd_dict = {}

    for col in num_cols:
        x1 = df.loc[df[group_col] == group1, col].dropna()
        x0 = df.loc[df[group_col] == group0, col].dropna()

        # 如果某一組完全沒有資料就跳過
        if len(x1) == 0 or len(x0) == 0:
            smd_dict[col] = np.nan
            continue

        m1, m0 = x1.mean(), x0.mean()
        s1, s0 = x1.std(), x0.std()
        sd_pooled = np.sqrt((s1**2 + s0**2) / 2)

        if sd_pooled == 0 or np.isnan(sd_pooled):
            smd = np.nan
        else:
            smd = (m1 - m0) / sd_pooled

        smd_dict[col] = smd

    smd_series = pd.Series(smd_dict, name='SMD')
    smd_df = smd_series.to_frame()
    smd_df['abs_SMD'] = smd_df['SMD'].abs()

    # 依 |SMD| 由大到小排序，方便看哪幾個最不平衡
    smd_df = smd_df.sort_values('abs_SMD', ascending=False)
    return smd_df

# 實際呼叫：對 external 以「死亡標記」0/1 分組做 SMD
smd_result = compute_smd_by_death(dfNew, group_col='死亡標記', group1=1, group0=0)

smd_result.head(20)  # 看前 20 個 SMD 最大的變數

,SMD,abs_SMD
觀察天數,-2.532427,2.532427
ADL_last_score,-0.851775,0.851775
ADL_Max,-0.690910,0.690910
ADL_總分_max,-0.690910,0.690910
ADL_first_score,-0.614117,0.614117
ADL_Min,-0.585312,0.585312
六個月內住院次數,0.532114,0.532114
BW_last,-0.377096,0.377096
BW_first,-0.360689,0.360689
ADL_std,-0.309496,0.309496


In [29]:
import numpy as np
import pandas as pd

def smd_for_categorical(df, col_name_non_death, col_name_death, label_col):
    """
    df: 包含類別名稱與兩組人數的 DataFrame
    col_name_non_death: 非死亡組欄位名稱
    col_name_death: 死亡組欄位名稱
    label_col: 類別欄位名稱（如 '地區'）
    """

    total_non = df[col_name_non_death].sum()
    total_death = df[col_name_death].sum()

    results = []

    for _, row in df.iterrows():
        name = row[label_col]
        x0 = row[col_name_non_death]
        x1 = row[col_name_death]

        p0 = x0 / total_non if total_non > 0 else 0  # 非死亡比例
        p1 = x1 / total_death if total_death > 0 else 0  # 死亡比例

        pooled = np.sqrt((p1*(1-p1) + p0*(1-p0)) / 2)

        smd = (p1 - p0) / pooled if pooled > 0 else np.nan

        results.append([name, x0, x1, p0, p1, smd])

    result_df = pd.DataFrame(results, columns=[
        label_col,
        col_name_non_death,
        col_name_death,
        "非死亡比例",
        "死亡比例",
        "SMD"
    ])

    return result_df


# --------------------------------------------
# 你的資料（可直接執行）
# --------------------------------------------
region_df = pd.DataFrame({
    '地區': ['北部','中部','南部','東部','離島'],
    '非死亡組': [1867, 3632, 12861, 269, 0],
    '死亡組': [378, 1122, 3742, 30, 0]
})

smd_region = smd_for_categorical(
    region_df,
    col_name_non_death='非死亡組',
    col_name_death='死亡組',
    label_col='地區'
)

In [30]:
smd_region

,地區,非死亡組,死亡組,非死亡比例,死亡比例,SMD
0,北部,1867,378,0.100220,0.071700,-0.101880
1,中部,3632,1122,0.194965,0.212822,0.044335
2,南部,12861,3742,0.690375,0.709788,0.042374
3,東部,269,30,0.014440,0.005690,-0.087737
4,離島,0,0,0.000000,0.000000,NaN
